In [1]:
# Break into timestamps of sentences/phrases

# Check for Names using google translate (using snippets from timestamps)

# Tag?

# determine team context

# determine sentiment 


# 

In [14]:
import pickle
import pandas as pd
import ast
from utils import normalize_string
import collections
import matplotlib.pyplot as plt
import matplotlib.dates as md
from fuzzywuzzy import fuzz
import seaborn as sns

In [4]:
# read in pickle 

path = '../data/text/raw/pickles/WOL_vs_ARS_2022-11-12.pickle'
              
with open(path, 'rb') as f:
    loaded_data = pickle.load(f)

print(loaded_data)

{'text': " in the Premier League. And away we go. Grenfords have done Arsenal a favour. But the value of the B's victory will be really felt if the Gunners can get their own win tonight. The Wolves, they'll hope to get off the bottom after they were put there by Forrest beating Christopher Hannis earlier. Yeah, never a nice day when you drop down without kicking a ball, but they've got a chance tonight, Wolves to change their fortunes. New manager in the stands can often have a positive effect. Need five at the back just to try and keep them solid, more solid base. Arsenal side playing with a swagger this season. Here's Saka. Here's Saka again. Here again. Benjamin White. Gabriel Tsinchenko returns to the Premier League side last week against Chelsea. Slug has never been away. Yeah, big bonus for the Gunners that him in the starting 11. Here's Gabriel. It's going to be the game, isn't it? Arsenal having most of the ball. Here's Granit Xhaka. He got a problem already. Feeling very well.

In [4]:
pd.DataFrame(loaded_data['segments'])

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,1.00,in the Premier League.,"[294, 264, 25194, 11199, 13]",0.0,-0.342801,1.534091,0.942406
1,1,0,2.40,3.40,And away we go.,"[400, 1314, 321, 352, 13]",0.0,-0.342801,1.534091,0.942406
2,2,0,3.40,5.60,Grenfords have done Arsenal a favour.,"[24913, 69, 5703, 362, 1096, 49156, 257, 8182,...",0.0,-0.342801,1.534091,0.942406
3,3,0,6.70,10.30,But the value of the B's victory will be real...,"[583, 264, 2158, 295, 264, 363, 311, 9812, 486...",0.0,-0.342801,1.534091,0.942406
4,4,0,10.40,12.80,if the Gunners can get their own win tonight.,"[498, 264, 14153, 2999, 393, 483, 641, 1065, 1...",0.0,-0.342801,1.534091,0.942406
...,...,...,...,...,...,...,...,...,...,...
1517,1517,576684,5782.84,5786.84,"The way this side is playing, hopefully givin...","[440, 636, 341, 1252, 307, 2433, 11, 4696, 290...",0.0,-0.173366,1.532020,0.000803
1518,1518,576684,5786.84,5790.84,"It's been a long time, hasn't it? 18 years si...","[467, 311, 668, 257, 938, 565, 11, 6132, 380, ...",0.0,-0.173366,1.532020,0.000803
1519,1519,579084,5790.84,5797.84,It's just that feel-good factor about the pla...,"[467, 311, 445, 300, 841, 12, 21753, 5952, 466...",0.0,-0.225968,1.401316,0.002143
1520,1520,579084,5797.84,5802.84,And the fans are absolutely loving it.,"[400, 264, 4499, 366, 3122, 9344, 309, 13]",0.0,-0.225968,1.401316,0.002143


In [6]:
events = pd.read_csv('../data/match_event_data.csv')
PL_teams = pd.read_csv('../data/pl_teams.csv',index_col=False)
PL_teams

,Team,Nicknames
0,AFC BOURNEMOUTH,The Cherries
1,ARSENAL,The Gunners
2,ASTON VILLA,"The Villans, The Villa"
3,BRENTFORD,"The Bees, The Reds"
4,BRIGHTON & HOVE ALBION,"The Seagulls, The Albion, Brighton"
5,CHELSEA,"The Blues, The Pensioners"
6,CRYSTAL PALACE,"The Eagles, The Glaziers"
7,EVERTON,"The Blues, The Toffees, The People's Club"
8,FULHAM,"Cottagers, Whites, Black and White army"
9,LEEDS UNITED,"The Whites, United, The Peacocks"


In [202]:
def count_entity_mentions(text,word_list):
    word_counts = collections.Counter()
    
    fuzzy_threshold = 80
    for word in word_list:
        if len(word) <4: fuzzy_threshold = 95
        for word_in_text in text.split():
            if fuzz.ratio(word, word_in_text) >= fuzzy_threshold:
                word_counts[word] += 1
    return word_counts

def get_word_frequency(text,df,team,i):
    text = normalize_string(text)
    text = text.upper()
    word_list = df[team+'_entities'][i]
    
    # Create a Counter object to count the occurrences of words in the text
    word_counts = count_entity_mentions(text, word_list)
    return word_counts




def plot_word_frequency(text, df,i):
    for team in ['Home','Away']:
        word_counts = get_word_frequency(text,df,team,i)

        word_counts = {k: v for k, v in sorted(word_counts.items(), key=lambda item: item[1],reverse = True)}
        # Plot the word frequencies using matplotlib
        plt.figure()
        plt.bar(word_counts.keys(), word_counts.values())
        plt.xlabel('Entity')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')
        plt.title(df['Home'][i]+' vs '+df['Away'][i]+':\n '+df[team][i]+' entity mentions',fontsize=10)
        plt.savefig('../data/graphs/team-entity-mentions-total/'+df['Home'][i]+' vs '+df['Away'][i]+':\n '+df[team][i]+'.png')
        plt.show()

    
def get_entity_frequency(df,EVENTS,i):
    entity_freq = time_entity_count(pd.DataFrame(df),EVENTS,i)
    entity_freq1 = entity_freq[['start','Home_entities','Away_entities']]
    entity_freq1['Away_entities'] *= -1 
    entity_freq1['start']= entity_freq1['start'].apply(lambda x: pd.to_datetime(x,unit='s'))

    home_test = entity_freq1[['start','Home_entities']].rename(columns={'Home_entities':'Entities'}).set_index('start').resample('3T').sum()
    away_test = entity_freq1[['start','Away_entities']].rename(columns={'Away_entities':'Entities'}).set_index('start').resample('3T').sum()


    home_test['Team'] = EVENTS['Home'][i]
    away_test['Team'] = EVENTS['Away'][i]

    entity_freq2 = home_test.append(away_test)

    entity_freq2 = entity_freq2.sort_values('start')
    entity_freq2.index = entity_freq2.index.strftime('%H:%M')

    fig, ax = plt.subplots(figsize = (20,10))    
    fig = sns.barplot(x = entity_freq2.index, y = "Entities",hue='Team', data = entity_freq2, 
                      estimator = sum, ax=ax, palette=['lightcoral','sandybrown'])
    ticks =  ax.get_yticks()
    ax.set_yticklabels([int(abs(tick)) for tick in ticks])
    plt.ylabel("Team Entity Mentions")
    plt.xlabel("Time (H:M)")
    plt.setp(plt.gca().xaxis.get_majorticklabels(),'rotation', 30)
    plt.savefig('../data/graphs/game-entity-mentions-total/'+EVENTS['Home'][i]+' vs '+EVENTS['Away'][i]+'.png')
    plt.show()

def process_text(text):
    text = normalize_string(text)
    text = text.upper()
    return text

        
def time_entity_count(segments_df,events,i):
    for team in ['Home','Away']:
        entity_list = events[team+'_entities'][i]
        segments_df[team+'_entities'] = segments_df['text'].apply(lambda x: sum(get_word_frequency(process_text(normalize_string(x)),events,team,i).values()))
    return segments_df
    
def get_pickle_file(events_df,i):
    pickle_path = '../data/text/raw/pickles/'
    name = events_df['1st_half_file'][i]
    date = events_df['Date'][i]
    pickle_file_name = name.split('-')[0] + date.split(' ')[0] + '.pickle'
    
    with open(pickle_path+pickle_file_name, 'rb') as f:
        data = pickle.load(f)
    
    return data

def read_entity_data():
    events = pd.read_csv('../data/match_event_data.csv')
    PL_teams = pd.read_csv('../data/pl_teams.csv',index_col=False)
    return events, PL_teams

def process_events(events):
    EVENTS = events.merge(PL_teams[['Team','Entities']], left_on='Home', right_on='Team', how='left').drop(columns = ['Team']).rename(columns = {'Entities':'Home_entities'})
    EVENTS['Home_names'] = EVENTS['Home_players'].apply(lambda x: [y[:][1].split()[-1] for y in ast.literal_eval(x)])
    EVENTS['Home_entities'] = EVENTS.apply(lambda x: x['Home_names'] + x['Home_entities'] , axis=1)
    EVENTS['Home_entities'] = EVENTS['Home_entities'].apply(lambda x: [normalize_string(y).upper() for y in x])
    EVENTS = EVENTS.drop(columns=['Home_names'])

    EVENTS = EVENTS.merge(PL_teams[['Team','Entities']], left_on='Away', right_on='Team', how='left').drop(columns = ['Team']).rename(columns = {'Entities':'Away_entities'})
    EVENTS['Away_names'] = EVENTS['Away_players'].apply(lambda x: [y[:][1].split()[-1] for y in ast.literal_eval(x)])
    EVENTS['Away_entities'] = EVENTS.apply(lambda x: x['Away_names'] + x['Away_entities'] , axis=1)
    EVENTS['Away_entities'] = EVENTS['Away_entities'].apply(lambda x: [normalize_string(y).upper() for y in x])
    EVENTS = EVENTS.drop(columns=['Away_names'])
    return EVENTS

def get_data(df,EVENTS,i):
    entity_freq = time_entity_count(pd.DataFrame(df),EVENTS,i)

    data = entity_freq[['start','text','Home_entities','Away_entities']]
    score = list(filter(lambda x: x.isdigit(),str(EVENTS.iloc[i]['Score'])))
    data['outcome'] = 'Home' if score[0]>score[1] else 'Away' if score[0]<score[1] else 'Draw'
    sentiment = pd.merge(data, pd.DataFrame(data['text'].apply(lambda x: analyzer.polarity_scores(x)).to_list()), left_index=True, right_index=True)
    return sentiment

def entity_analysis(plot_totals=False,plot_frequency=False):
    # read data
    events, PL_teams = read_entity_data()
    
    # process events
    events = process_events(events)
    
    data_list = []
    outcome_list = []
    for game in range(len(events)):
        # get commentary
        commentary_dict = get_pickle_file(events,game)
        
        #plot word frequency
        if plot_totals == True:
            plot_word_frequency(commentary_dict['text'],EVENTS,game) 
            
        if plot_frequency==True:
            get_entity_frequency(commentary_dict['segments'],EVENTS,game)
            
        data_entry = get_data(loaded_data['segments'],EVENTS,game)
        data_list.append(data_entry)
       
    return data_list
# plot_word_frequency(loaded_data['text'],EVENTS)


In [203]:
data = entity_analysis()

/var/folders/kp/8md06y697yvbxpk_5yrygl4c0000gn/T/ipykernel_88626/1944437076.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['outcome'] = 'Home' if score[0]>score[1] else 'Away' if score[0]<score[1] else 'Draw'
/var/folders/kp/8md06y697yvbxpk_5yrygl4c0000gn/T/ipykernel_88626/1944437076.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['outcome'] = 'Home' if score[0]>score[1] else 'Away' if score[0]<score[1] else 'Draw'
/var/folders/kp/8md06y697yvbxpk_5yrygl4c0000gn/T/ipykernel_88626/19444

In [217]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Concatenate the DataFrames in the list along the rows
df = pd.concat(data)

# Extract the feature and target variables
X = df.drop(['outcome','text'], axis=1)
y = df['outcome']

replace_dict = {'Home':0,'Away':1,'Draw':2}
y = y.map(replace_dict)



# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize the logistic regressor
clf = LogisticRegression()

# Train the model on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)


Accuracy:  0.3358862144420131


In [227]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import numpy as np

# Concatenate the DataFrames in the list along the rows
df = pd.concat(data)

# Extract the feature and target variables
X = df.drop(['outcome','text'], axis=1)
y = df['outcome']

replace_dict = {'Home':0,'Away':1,'Draw':2}
y = y.map(replace_dict)

# Reshape the features for LSTM
X = np.reshape(X.values, (X.shape[0], X.shape[1], 1))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Initialize the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train, y_train, epochs=5, batch_size=32)

# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)

# Calculate the accuracy of the model
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

Epoch 1/5
115/115 [==============================] - 1s 3ms/step - loss: 0.0459 - accuracy: 0.3357
Epoch 2/5
115/115 [==============================] - 0s 2ms/step - loss: -0.0109 - accuracy: 0.3357
Epoch 3/5
115/115 [==============================] - 0s 2ms/step - loss: -0.0157 - accuracy: 0.3357
Epoch 4/5
115/115 [==============================] - 0s 2ms/step - loss: -0.0283 - accuracy: 0.3357
Epoch 5/5
29/29 [==============================] - 0s 853us/step
Accuracy:  0.3238512035010941


(914, 1)

In [168]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#Build dataset
i=0
df = loaded_data['segments']


entity_freq = time_entity_count(pd.DataFrame(df),EVENTS,i)
data = entity_freq[['start','text','Home_entities','Away_entities']]
score = list(filter(lambda x: x.isdigit(),str(EVENTS.iloc[i]['Score'])))
data['outcome'] = 'Home' if score[0]>score[1] else 'Away' if score[0]<score[1] else 'Draw'
sentiment = pd.merge(data, pd.DataFrame(data['text'].apply(lambda x: analyzer.polarity_scores(x)).to_list()), left_index=True, right_index=True)

    

/var/folders/kp/8md06y697yvbxpk_5yrygl4c0000gn/T/ipykernel_88626/189300626.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['outcome'] = 'Home' if score[0]>score[1] else 'Away' if score[0]<score[1] else 'Draw'


In [172]:
sentiment[['Home_entities','Away_entities']].value_counts()

Home_entities  Away_entities
0              0                1023
               1                 403
1              0                  46
0              2                  21
2              0                  15
1              1                   7
0              3                   2
2              1                   2
1              2                   1
               3                   1
3              0                   1
dtype: int64

In [ ]:
# Add surnames to entity lists for away
# Add surnames to entity lists for home
# remove names that are common to both teams
# If two hits back to back then remove 1

In [ ]:
# PL_teams.to_csv('../data/PL_teams.csv',index=False)

In [1]:
# Belfast
Salary = 32000.00
take_home = 25164.16

rent_bills = 600 * 12
leftover = take_home - rent_bills
leftover

17964.16

In [2]:
# Thames
Salary = 27500
take_home = 22212.16

rent_bills = 400 * 12
leftover = take_home - rent_bills
leftover

17412.16

In [10]:
#Putney
Salary = 37000
take_home = 29182.40

rent_bills = 750 * 12
leftover = take_home - rent_bills
leftover

20182.4

In [3]:
# Santander
Salary = 30000
take_home = 23612.16

rent_bills = 500 * 12
leftover = take_home - rent_bills
leftover

17612.16

In [4]:
# dmg
Salary = 50000
take_home = 34932.16

# Salary = 40000
# take_home = 29272.16

rent_bills = 7000 + (800*12)
leftover = take_home - rent_bills
leftover

18332.160000000003

In [5]:
#Marketcast
Salary = 68000
take_home = 44036.86

rent_bills = 3000 + (800*12) 
leftover = take_home - rent_bills


yearly_cost = 1000 * 12

savings = leftover - yearly_cost 
print('takehome after bills',leftover)
print('savings',savings)

takehome after bills 31436.86
savings 19436.86


In [ ]:
1800 + 400